In [1]:
#-*-coding:utf-8-*- #

# 准备工作，导入库
# import library
import logging
import math
import random
import mxnet as mx
import numpy as np

In [3]:
# 打开调试信息的显示
logging.getLogger().setLevel(logging.DEBUG) 

In [4]:
# 设置超参数
n_sample = 1000 # sample data number
batch_size = 1 # batch size
learning_rate = 0.1 # learning rate
n_epoch = 1 # epoch number

In [5]:
# 获得训练数据
# 每个数据点是在 (0,1) 之间的 2 个随机数
train_in = [[ random.uniform(0, 1) for c in range(2)] for n in range(n_sample)] 

# 期望输出，先初始化为 0
train_out = [0 for n in range(n_sample)]

In [6]:
for i in range(n_sample):
    # 每个数据点的期望输出是 2 个输入数中的大者
    train_out[i] = max(train_in[i][0], train_in[i][1])

In [7]:
# 训练数据的迭代器
train_iter = mx.io.NDArrayIter(data = np.array(train_in), label = {'reg_label':np.array(train_out)}, batch_size = batch_size, shuffle = True)

# 定义网络结构   输入层
src = mx.sym.Variable('data')

# 全连接层  num_hidden->此层有一个神经元
fc  = mx.sym.FullyConnected(data = src, num_hidden = 1, name = 'fc')

# 输入数据为fc, 命名为reg
net = mx.sym.LinearRegressionOutput(data = fc, name = 'reg')

# 定义变量module为需训练的网络模组
module = mx.mod.Module(symbol = net, label_names = (['reg_label']))

In [8]:
# 定义epoch_callback函数，在此它负责输出训练出的网络的参数值

def epoch_callback(epoch, symbol, arg_params, aux_params):
    for k in arg_params: # 对于所有参数…
        print(k) # 输出参数名
        print(arg_params[k].asnumpy()) # 参数值，转为 NumPy 数组，输出更美观

        # 调用module.fit 开始训练        
module.fit(
    train_iter, # 训练数据的迭代器
    eval_data = None, # 在此只训练，不使用测试数据
    eval_metric = mx.metric.create('mse'), # 输出 MSE 损失信息
    optimizer = 'sgd', # 梯度下降算法为 SGD
    # 设置学习速率
    optimizer_params = {'learning_rate': learning_rate}, 
    num_epoch = n_epoch, # 训练 epoch 数
    # 每经过 100 个 batch 输出训练速度 
    batch_end_callback = mx.callback.Speedometer(batch_size, 100), 
    epoch_end_callback = epoch_callback, # 完成每个 epoch 后调用 epoch_callback
)

INFO:root:Epoch[0] Batch [100]	Speed: 2475.20 samples/sec	mse=0.042577
INFO:root:Epoch[0] Batch [200]	Speed: 2432.91 samples/sec	mse=0.014249
INFO:root:Epoch[0] Batch [300]	Speed: 2324.71 samples/sec	mse=0.017389
INFO:root:Epoch[0] Batch [400]	Speed: 2528.38 samples/sec	mse=0.014808
INFO:root:Epoch[0] Batch [500]	Speed: 2519.98 samples/sec	mse=0.012631
INFO:root:Epoch[0] Batch [600]	Speed: 2462.15 samples/sec	mse=0.015068
INFO:root:Epoch[0] Batch [700]	Speed: 2379.48 samples/sec	mse=0.015255
INFO:root:Epoch[0] Batch [800]	Speed: 2372.04 samples/sec	mse=0.017084
INFO:root:Epoch[0] Batch [900]	Speed: 2453.40 samples/sec	mse=0.016849
INFO:root:Epoch[0] Train-mse=0.014489
INFO:root:Epoch[0] Time cost=0.710


fc_weight
[[ 0.55011141  0.49512723]]
fc_bias
[ 0.14131179]
